In [40]:
import cv2
import numpy as np
import os
import random
from PIL import Image

np.random.seed(0)
random.seed(0)
font_dir = r'C:\Windows\Fonts'
# font_list = [os.path.basename(f) for f in os.listdir(font_dir) if f.endswith('.ttf')]
# font_list= font_list[:30]
font_list=['FONT_HERSHEY_COMPLEX', 'FONT_HERSHEY_COMPLEX_SMALL', 'FONT_HERSHEY_DUPLEX', 'FONT_HERSHEY_PLAIN', 'FONT_HERSHEY_SCRIPT_COMPLEX', 'FONT_HERSHEY_SCRIPT_SIMPLEX', 'FONT_HERSHEY_SIMPLEX', 'FONT_HERSHEY_TRIPLEX', 'FONT_ITALIC']

# 画像のサイズを設定する
pixel = 60
N,M=10,10
K=500
dir_name='numbers_ln'

for id in range(K):
    # 画像を作成する
    image = np.zeros((pixel*N, pixel*M, 3), np.uint8)
    image.fill(255)

    # 数字をランダムに選択する
    numbers = [[random.randint(0, 9) for _ in range(M)] for _ in range(N)]

    # 数字を描画する
    for i in range(N):
        for j in range(M):
            # フォントをランダムに選択する
            # フォントサイズもランダムに上下させる
            font_size = 1+np.random.lognormal(0,1)/5
            # font_path = os.path.join(font_dir, random.choice(font_list))
            # font = cv2.FONT_HERSHEY_SIMPLEX
            font_name=random.choice(font_list)
            font = getattr(cv2,font_name)

            
            # 数字を描画する
            number = numbers[i][j]

            text_size, _ = cv2.getTextSize(str(number), font, font_size, thickness=1)
            text_width, text_height = text_size
            x = j*pixel + (pixel - text_width) // 2
            y = i*pixel + (pixel + text_height) // 2
            
            # 中心から上下左右に正規分布でずらす
            mu, sigma = 0, 3
            x += int(np.random.normal(mu, sigma))
            y += int(np.random.normal(mu, sigma))
            
            if number:
                cv2.putText(image, str(number), (x, y), font, font_size, (0, 0, 0), thickness=2,lineType=cv2.LINE_AA)
    
    # 線を引く
    H,W,_=image.shape
    sigma_l=1
    for num,i in enumerate(range(0, H+1, H//N)):
        if num%3==0:    
            cv2.line(image, (i+int(np.random.normal(0,sigma_l)), 0), (i+int(np.random.normal(0,sigma_l)), H), (0, 0, 0), thickness=4, lineType=cv2.LINE_AA)
        else:
            cv2.line(image, (i+int(np.random.normal(0,sigma_l)), 0), (i+int(np.random.normal(0,sigma_l)), H), (0, 0, 0), thickness=1)
    for num,j in enumerate(range(0, W+1, W//M)):
        if num%3==0:
            cv2.line(image, (0, j+int(np.random.normal(0,sigma_l))), (W, j+int(np.random.normal(0,sigma_l))), (0, 0, 0), thickness=4, lineType=cv2.LINE_AA)
        else:
            cv2.line(image, (0, j+int(np.random.normal(0,sigma_l))), (W, j+int(np.random.normal(0,sigma_l))), (0, 0, 0), thickness=1)
    
    # 微小に回転させる
    image=cv2.bitwise_not(image)
    angle=random.uniform(-1.5,1.5)
    H,W,_=image.shape
    center=(0,0)
    Mat=cv2.getRotationMatrix2D(center,angle,1.0)
    image=cv2.warpAffine(image,Mat,(H,W))
    image=cv2.bitwise_not(image)
    # print(type(image))
    # image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    # image=Image.fromarray(image)

    # 画像を保存する
    cv2.imwrite(f'./training_data/{dir_name}/numbers_{id}.png',image)
    # numberを保存する
    np.save(f'./training_data/{dir_name}/numbers_{id}.npy', numbers)